In [9]:
from darts import TimeSeries
import datetime
import pandas as pd
from darts.models import (
    RNNModel,
    TCNModel,
    TransformerModel,
    NBEATSModel,
    BlockRNNModel,
    VARIMA,
)

from darts.dataprocessing.transformers import MissingValuesFiller
data_filler = MissingValuesFiller()

In [10]:
import logging
import warnings

logging.disable(logging.CRITICAL)
warnings.filterwarnings("ignore")

In [11]:
df = pd.read_csv('../data/processed/R80711_imputed.csv')
cols = ['Ba_avg', 'P_avg', 'Q_avg', 'Ya_avg', 'Yt_avg', 'Ws1_avg', 'Ws2_avg',
        'Ws_avg', 'Wa_avg', 'Va_avg', 'Ot_avg', 'Rs_avg', 'Rbt_avg', 'Rm_avg','temp', 
        'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_1h','snow_1h']
df['Date_time'] = pd.to_datetime(df['Date_time'], utc=True).dt.tz_localize(None)

In [12]:
series = TimeSeries.from_dataframe(df, time_col='Date_time', value_cols=cols, fill_missing_dates=True, freq=None)
series = data_filler.transform(series)
train, val = series.split_after(pd.Timestamp('2017-01-01 00:00:00'))

In [12]:
model= TransformerModel(
    input_chunk_length=500, 
    output_chunk_length=216, 
    n_epochs=2, 
    random_state=0, 
    batch_size=32,
    pl_trainer_kwargs={
      "accelerator": "gpu",
      "devices": [0]
    },
    log_tensorboard=True,
)

In [8]:
model.fit(train, val_series=val, verbose=True, num_loader_workers=4)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ ❱ 1 model.fit(train, val_series=val, verbose=True, num_loader_workers=4)                         │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

In [33]:
df = pd.read_csv('../data/imputed/R80711.csv')
df['Date_time'] = pd.to_datetime(df['Date_time'], utc=True).dt.tz_localize(None)
df['DT_dummy'] = df.Date_time.copy()
df = df.set_index('DT_dummy')

In [55]:
df['year'] = df.index.year
df['time_of_day'] = df.index.hour*60 + df.index.minute
df['month'] = df.index.month
df['dayofyear'] = df.index.dayofyear
df['weekofyear'] = df.index.weekofyear
df['hour'] = df.index.hour

In [56]:
df.head()

,Date_time,Ba_avg,P_avg,Q_avg,Ya_avg,Yt_avg,Ws1_avg,Ws2_avg,Ws_avg,Wa_avg,...,humidity,wind_speed,wind_deg,rain_1h,snow_1h,year,time_of_day,month,dayofyear,weekofyear
DT_dummy,,,,,,,,,,,,,,,,,,,,,
2012-12-31 23:00:00,2012-12-31 23:00:00,-1.00,1072.6500,14.490000,161.44000,12.730000,8.770001,9.160000,8.960000,165.25000,...,75.0,5.66,180.0,0.0,0.0,2012,1380,12,366,1
2012-12-31 23:10:00,2012-12-31 23:10:00,-1.00,1061.4301,23.700001,161.44000,14.990000,8.660000,9.120000,8.890000,168.28999,...,75.0,5.66,180.0,0.0,0.0,2012,1390,12,366,1
2012-12-31 23:20:00,2012-12-31 23:20:00,-1.00,1144.7900,25.480000,161.44000,21.750000,8.940000,9.450000,9.190000,170.08000,...,75.0,5.66,180.0,0.0,0.0,2012,1400,12,366,1
2012-12-31 23:30:00,2012-12-31 23:30:00,-1.00,1183.9800,24.379999,173.41000,23.799999,8.870000,8.979999,8.920000,175.33000,...,75.0,5.66,180.0,0.0,0.0,2012,1410,12,366,1
2012-12-31 23:40:00,2012-12-31 23:40:00,-0.98,1317.5500,14.470000,177.89999,17.910000,9.440000,9.510000,9.479999,173.07001,...,75.0,5.66,180.0,0.0,0.0,2012,1420,12,366,1


In [61]:
df.index[100000].weekofyear

48